# Segmentation and motion correction of in-vivo recordings

Before running this notebook, generate an ROI called `correctionReference.npy` using [Robopy](https://github.com/fedeceri85/robopy2). This ROI should be placed in the same folder as the recording and contain a trace where the "jumps" are visible as decreases (ideally) or increases in fluorescence.

This notebook contains the preprocessing steps and segmentation routines for in vivo recordings. Some utilities used in this notebook are:

- `thorlabsFile` from `movieTools`: This class contains the actual recording data, functions to smooth and filter it, and display it in a Napari viewer. The `thorlabsFile` object is passed to other routines to extract data. When loading files, they are automatically smoothed with a 3D Gaussian kernel (2x2x2).
- `jumpFramesFinder` from `visualisationTools`: This displays an ipywidget interface with tools to remove out-of-focus frames. This interface displays the `correctionReference` generated using Robopy (first step of the analysis).
- `jupyterpy` from `movieTools`: This interface connects to the currently displayed `thorlabsFile` and contains buttons for the segmentation of hair cells, calcium waves, and fibers.
- `maskMatching` from `movieTools`: This is an ipywidgets/Napari interface that combines repetitive recordings of the same field of view to generate a `matchingMask`, i.e., a label image where hair cell ROIs are color-coded with the same color.


In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%pylab
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import os
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy.signal import argrelextrema,argrelmin,argrelmax
import sys
sys.path.append('../../src')

import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, Dropdown
import visualisationTools as vu
import traceUtilities as tu
import ast 
from movieTools import thorlabsFile

parameterFolder = '../../parameters/'
fileHeader = 'NeuroD_SGN' #SNAP25_SGN

inputFilename = os.path.join('../../',fileHeader)+'.xlsx'
corrFilename = os.path.join(parameterFolder,fileHeader)+ '_with_corr_params.csv'
jumpFrameFilename = os.path.join(parameterFolder,fileHeader+'jump_frames.csv')
jumpFrameMaxFilename = os.path.join(parameterFolder,fileHeader+'jumpMax_frames.csv')
correctionReferenceTraceFile = os.path.join(parameterFolder,fileHeader+'correctionReference.csv')

First we load all the excel files above as pandas dataframes, or create an empty dataframe if they don't exist

In [3]:
alldata = pd.read_excel(inputFilename)
alldata = alldata[alldata['discard']!=1]

try:
    corr_param = pd.read_csv(corrFilename)
except:
    print('No jump correction parameter file')

try:
    allminima = pd.read_csv(jumpFrameFilename)
except FileNotFoundError:
    allminima = pd.DataFrame()

try:
    allmaxima = pd.read_csv(jumpFrameMaxFilename)
except FileNotFoundError:
    allmaxima = pd.DataFrame()


try:
    correctionReferenceTrace = pd.read_csv(correctionReferenceTraceFile)
except FileNotFoundError:
    correctionReferenceTrace = pd.DataFrame()

Then we go through the list of recordings and compare it to the list of parameters. We copy over the already calculated parameters for all the files that have already been processed, and use default (blank) values for all the new files.

In [4]:
for el in alldata['Folder'].unique():
    try:
        alldata.loc[alldata['Folder']==el,'Window left'] = corr_param.loc[corr_param['Folder']==el,'Window left'].values[0]
        alldata.loc[alldata['Folder']==el,'Window right'] = corr_param.loc[corr_param['Folder']==el,'Window right'].values[0]
        alldata.loc[alldata['Folder']==el,'Minima order'] = corr_param.loc[corr_param['Folder']==el,'Minima order'].values[0]



    except (IndexError, NameError, KeyError):
        print('Could not find correction parameters for '+ el+' Setting to default.')
        alldata.loc[alldata['Folder']==el,'Window left'] = 0
        alldata.loc[alldata['Folder']==el,'Window right'] = 0
        alldata.loc[alldata['Folder']==el,'Minima order'] = 50 



    try:
        alldata.loc[alldata['Folder']==el,'Window Max left'] = corr_param.loc[corr_param['Folder']==el,'Window Max left'].values[0]
        alldata.loc[alldata['Folder']==el,'Window Max right'] = corr_param.loc[corr_param['Folder']==el,'Window Max right'].values[0]
        alldata.loc[alldata['Folder']==el,'Maxima order'] = corr_param.loc[corr_param['Folder']==el,'Maxima order'].values[0]
    except (IndexError, NameError, KeyError):
        alldata.loc[alldata['Folder']==el,'Window Max left'] = 0
        alldata.loc[alldata['Folder']==el,'Window Max right'] = 0
        alldata.loc[alldata['Folder']==el,'Maxima order'] = 50

alldata.loc[alldata['Window left'].isna(),'Window left'] = 0
alldata.loc[alldata['Window right'].isna(),'Window right'] = 0
alldata.loc[alldata['Minima order'].isna(),'Minima order'] = 50

alldata.loc[alldata['Folder'].isna(),'Window Max left'] = 0
alldata.loc[alldata['Folder'].isna(),'Window Max right'] = 0
alldata.loc[alldata['Folder'].isna(),'Maxima order'] = 50

alldata['Window left'] = alldata['Window left'].astype(int)
alldata['Window right'] = alldata['Window right'].astype(int)
alldata['Minima order'] = alldata['Minima order'].astype(int)

alldata['Window Max left'] = alldata['Window Max left'].astype(int)
alldata['Window Max right'] = alldata['Window Max right'].astype(int)
alldata['Maxima order'] = alldata['Maxima order'].astype(int)


alldata.loc[alldata['Minima order']==1,'Minima order'] = 50
alldata.loc[alldata['Maxima order']==1,'Maxima order'] = 50

try:
    for el in alldata['Folder'].unique():
        alldata.loc[alldata['Folder']==el,'ExtraCorrectionIntervals'] = corr_param.loc[corr_param['Folder']==el,'ExtraCorrectionIntervals'].values[0]
except:
    pass

for j,el in alldata.iterrows():
    try:
        alldata.at[j,'ExtraCorrectionIntervals'] = ast.literal_eval(alldata.at[j,'ExtraCorrectionIntervals'] )
    except (ValueError,KeyError):
         alldata.at[j,'ExtraCorrectionIntervals'] = np.nan
alldata['ExtraCorrectionIntervals'] =  alldata['ExtraCorrectionIntervals'].astype('object')


try:
    for el in alldata['Folder'].unique():
        alldata.loc[alldata['Folder']==el,'TemplateIntervals'] = corr_param.loc[corr_param['Folder']==el,'TemplateIntervals'].values[0]
except:
    pass

for j,el in alldata.iterrows():
    try:
        alldata.at[j,'TemplateIntervals'] = ast.literal_eval(alldata.at[j,'TemplateIntervals'] )
    except (ValueError,KeyError):
         alldata.at[j,'TemplateIntervals'] = np.nan

alldata['TemplateIntervals'] =  alldata['TemplateIntervals'].astype('object')

In [ ]:
master = alldata.copy().reset_index()

# 1- Jump Correction. 
Remember to run the block after this every now and then to save the results in case this notebook crashes
`jumpFramesFinder` is a comprehensive tool that combines interactive widgets, data processing, and visualization to assist users in identifying and correcting frame jumps in their datasets.
The jumpFramesFinder function is designed to facilitate the identification and correction of frame jumps in a in vivo movie. This function utilizes a variety of widgets from the `ipywidgets` library to create an interactive user interface  that allows users to adjust parameters and visualize the effects of these adjustments in real-time.

The user can utilise various sliders to control various parameters such as the trace number, window sizes for minima and maxima detection, and the order of minima and maxima. These sliders allow users to fine-tune the detection of frame jumps by adjusting the sensitivity and range of the detection algorithm.

Users can perform specific actions, such as loading original or corrected movie, saving processed files, and creating templates for further analysis.

The function also sets up a plot using `plotly.graph_objects` to visualize the original and corrected traces, as well as the detected jumps. This visualization helps users to see the impact of their adjustments and make decisions about the parameters they set.

The corrected movie is visualised through the `thorlabsFile` object in a napari window.

After this step, launch the `batchModtionCorrect.ipynb` notebook to motion correct the "jumpCorrected.tif" files, which will be saved as `jumpCorrected-mc.tif` files. After that, come back to this notebook, and run this block to load the motion corrected files, to be used for segmentation.

In [ ]:
tb = thorlabsFile()
vu.jumpFramesFinder(master,allminima,allmaxima,correctionReferenceTrace,tb)

## Save parameters block. Uncomment it and run it while doing the previous step to prevent data loss.

In [ ]:
#master.to_csv(corrFilename)
#allminima.to_csv(jumpFrameFilename)
#allmaxima.to_csv(jumpFrameMaxFilename)
#correctionReferenceTrace.to_csv(correctionReferenceTraceFile)

## The next block displays the jupyterpy interface. This connects to the opened image in napari (the current thorlabsFile object, it should be loaded using the jumpFramesFinder above) and provides buttons with specific analysis steps for each type of experiment.  

The jupyterPy function is designed to create an interactive user interface (UI) for analyzing image data within a Jupyter notebook. This function utilizes the ipywidgets library to create various interactive widgets, such as buttons, sliders, and text inputs, which are then organized into a tabbed layout for different analysis tasks.
The istance of jupyterPy connects to the movie displayed in the existing napari windows (passed through the thorlabsFile object.)

The software allows to perform specific segmentation analysis for IHCs, Calcium waves, and afferent terminals. 
For IHCs, the software uses cellpose to automatically identify the cell bodies. The segmentation is shown as a `label` layer (named 'Masks') in Napari. Users can modify the existing ROIs using Napari tools (e.g, splitting ROIs). The function also allows to plot and save the fluorescence profile of the individual ROIs. 


In [ ]:
from movieTools import jupyterPy
JP = jupyterPy(tb)

In [10]:
#Check that the masks are sequentials and that there are no sgorbietti in the ihcs
# These are not corrected at the moment, but keep in mind that a few recordings have small or non sequential rois

In [12]:
from skimage.io import imread
from skimage import measure
allareas = []
for i,el in enumerate(master['Folder'].unique()):
   # print(el)
    try:
        mask = imread('D'+el[1:]+'\\processedMovies\\Masks.tif')
        
        props = measure.regionprops(mask)
        labels = [p.label for p in props]
        areas = [p.area for p in props]
        centroid_x = [p.centroid[0] for p in props]
        centroid_y = [p.centroid[1] for p in props]
        allareas.extend(areas)
        #Check small rois
        for area in areas:
            if area<100:
                print('Small ROI:{} {}'.format(i,el))
        #Check if labels are sequential
        if sum(diff(labels)<=0):
            print('Labels not sequential:{} {}'.format(i,el))
        
        #Check missing rois
        if max(labels)!=len(labels):
            print('Missing rois: {} {}'.format(i,el))
            
        #Check non sequential ROIs
        if sum(diff(centroid_y)<=0):
            print('ROI positions not sequential:{} {}'.format(i,el))
        
    except FileNotFoundError:
        pass

In [ ]:
tb.app.add_image(avg)

# 2- Match ROIs from different recordings

In [ ]:
#Change drive 
localDrive = 'E'
master['Folder'] = localDrive + master['Folder'].str[1:]

In [7]:
from movieTools import maskMatching,extractImagesMaskMatching

In [ ]:
maskMatching(master,onlyIHCs=False)

# 3- Calculate the local pixel correlation
This block calculates how much the pixels belonging to the same ROI are correlated. This helps with peaks identification, as peaks resulting from artifacts (e.g., cross-talk from a neighboring cell) will have a low correlation and can be rejected on this basis
The correlation traces are saved in the same folder as the traces (processedMovies folder inside the experiment folder)

In [ ]:
for j,el in master.iterrows():
    
    if not os.path.exists(os.path.join(el['Folder'],'processedMovies','traces_localCorr.csv')):
        print(el['Folder'])
        print(j)
        try:
            df = tu.calculatePixelRollingCorr(el['Folder'],int(el['fps']/2),8,addNoise=True,maskfilename='SGN ROIs.tif')
            df.to_csv(os.path.join(el['Folder'],'processedMovies','traces_localCorr.csv'))
        except FileNotFoundError:
            print('fnf')
            

# N numbers

In [ ]:
master.groupby(['Age','Mouse ID']).count()['Folder']